https://en.wikipedia.org/wiki/Tf%E2%80%93idf

In [1]:
import math
import os
import fnmatch # https://docs.python.org/3/library/fnmatch.html

Source https://stevenloria.com/tf-idf/ <BR>

Caveat: this post now uses TextBlob for breaking up the text into words and getting the word counts.

In [2]:
def term_freq(term, list_of_words_in_document):
    """
    computes "term frequency" which is the number of times a word appears in a document, 
    normalized by dividing by the total number of words in document. 
    """
    return list_of_words_in_document.count(term)/(len(list_of_words_in_document)*1.0)

def number_of_documents_containing(term,all_documents):
    """
    Returns the number of documents containing word. 
    """
    countr=0
    for this_doc in all_documents:
        if (term in this_doc):
            countr+=1
    return countr

def inverse_doc_freq(term, all_documents):
    """
    computes "inverse document frequency" which measures 
    how common a word is among all documents in corpus. 
    The more common a word is, the lower its idf. 
    Take the ratio of the total number of documents to the number of documents containing word, 
    then take the log of that. Add 1 to the divisor to prevent division by zero.
    """
    return math.log(len(all_documents) / ( 1.0 + number_of_documents_containing(term, all_documents)))

def tfidf(term, list_of_words_in_document, all_documents):
    """
    computes the TF-IDF score. It's the product of tf and idf.
    """
    return term_freq(term, list_of_words_in_document) * inverse_doc_freq(term, all_documents)

The \*.dat files in the directory have only key words from each file

Convert the .dat contents to lists per document

In [3]:
all_documents={}
all_words_from_all_docs=[]
all_terms=[]
foldr='data/'
fname='*.dat'
for file_name in os.listdir(foldr):
    #print(file_name)
    if fnmatch.fnmatch(file_name, fname): # Unix shell-style wildcards
        print(file_name)
        with open(foldr+file_name,'r') as fil:
            words_in_file=fil.read().split("\n")
        # remove empty strings from list of words
        while "" in words_in_file:
            words_in_file.remove("")
        # save the words per file as value in a dictionary
        all_documents[file_name]=words_in_file
        print('has',len(words_in_file),'words\n')
        # also save all the words to a list
        for this_word in words_in_file:
            all_words_from_all_docs.append(this_word)
            

week1_50 years of data science v2.pdf.dat
has 90 words

week1_a Very Short History Of Data Science_1.docx.dat
has 96 words

week1_assignment 1 Summary.pdf.dat
has 110 words

week1_380.txt.dat
has 106 words

week1_A Very Short History Of Data Science.docx.dat
has 94 words

week1_50 Years Data Science Summary.docx.dat
has 133 words



In [4]:
len(all_documents)

6

In [5]:
all_words_from_all_docs = list(set(all_words_from_all_docs))
len(all_words_from_all_docs)

482

Sample sizes are small, so results are not reliable representations of the document

In [6]:
for doc_name, word_list_in_this_doc in all_documents.items():
    if (len(word_list_in_this_doc)==0):
        print("error: empty input file"+doc_name)
    else:
        dic_of_terms={}
        for this_term in word_list_in_this_doc:
            dic_of_terms[this_term] = tfidf(this_term, word_list_in_this_doc, all_words_from_all_docs)
        #print(dic_of_terms)
        print('\n'+doc_name)
        terms_in_doc_sorted_by_score=sorted(dic_of_terms.items(), key=lambda x: x[1], reverse=True)
        # first 40 words by importance
        for this_tup in terms_in_doc_sorted_by_score[0:40]:
            print(this_tup)
#        for indx in range(10):
#            print(terms_in_doc_sorted_by_score[indx][0] + ":"+str(terms_in_doc_sorted_by_score[indx][1]))


week1_50 years of data science v2.pdf.dat
('princeton', 0.060942188149896166)
('interesting', 0.060942188149896166)
('larger', 0.060942188149896166)
('summary', 0.060942188149896166)
('training', 0.060942188149896166)
('version', 0.060942188149896166)
('programs', 0.060942188149896166)
('successes', 0.060942188149896166)
('important', 0.060942188149896166)
('caused', 0.060942188149896166)
('knowledge', 0.060942188149896166)
('demand', 0.060942188149896166)
('interpreting', 0.060942188149896166)
('value', 0.060942188149896166)
('fit', 0.060942188149896166)
('analyzing', 0.060942188149896166)
('centennial', 0.060942188149896166)
('would', 0.060942188149896166)
('finally', 0.060942188149896166)
('necessary', 0.060942188149896166)
('volumes', 0.060942188149896166)
('described', 0.060942188149896166)
('collecting', 0.060942188149896166)
('skills', 0.060942188149896166)
('sprung', 0.060942188149896166)
('new', 0.060942188149896166)
('across', 0.060942188149896166)
('networking', 0.060942188

In [55]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.feature_extraction import text
from sklearn.metrics.pairwise import cosine_similarity

# https://www.kaggle.com/sameersmahajan/people-wikipedia-data

In [8]:
# text.ENGLISH_STOP_WORDS

In [9]:
df = pd.read_csv('./data/people_wiki.csv')

In [10]:
df.shape

(42786, 3)

In [11]:
df.head()

,URI,name,text
0,<http://dbpedia.org/resource/Digby_Morrell>,Digby Morrell,digby morrell born 10 october 1979 is a former...
1,<http://dbpedia.org/resource/Alfred_J._Lewy>,Alfred J. Lewy,alfred j lewy aka sandy lewy graduated from un...
2,<http://dbpedia.org/resource/Harpdog_Brown>,Harpdog Brown,harpdog brown is a singer and harmonica player...
3,<http://dbpedia.org/resource/Franz_Rottensteiner>,Franz Rottensteiner,franz rottensteiner born in waidmannsfeld lowe...
4,<http://dbpedia.org/resource/G-Enka>,G-Enka,henry krvits born 30 december 1974 in tallinn ...


In [12]:
df['text'][0]

'digby morrell born 10 october 1979 is a former australian rules footballer who played with the kangaroos and carlton in the australian football league aflfrom western australia morrell played his early senior football for west perth his 44game senior career for the falcons spanned 19982000 and he was the clubs leading goalkicker in 2000 at the age of 21 morrell was recruited to the australian football league by the kangaroos football club with its third round selection in the 2001 afl rookie draft as a forward he twice kicked five goals during his time with the kangaroos the first was in a losing cause against sydney in 2002 and the other the following season in a drawn game against brisbaneafter the 2003 season morrell was traded along with david teague to the carlton football club in exchange for corey mckernan he played 32 games for the blues before being delisted at the end of 2005 he continued to play victorian football league vfl football with the northern bullants carltons vfla

In [31]:
df['text']

0        digby morrell born 10 october 1979 is a former...
1        alfred j lewy aka sandy lewy graduated from un...
2        harpdog brown is a singer and harmonica player...
3        franz rottensteiner born in waidmannsfeld lowe...
4        henry krvits born 30 december 1974 in tallinn ...
                               ...                        
42781    motoaki takenouchi born july 8 1967 saitama pr...
42782    alan graham judge born 14 may 1960 is a retire...
42783    eduardo lara lozano born 4 september 1959 in c...
42784    tatiana faberg is an author and faberg scholar...
42785    kenneth thomas born february 24 1938 was chief...
Name: text, Length: 42786, dtype: object

In [56]:
vectorizer = CountVectorizer(
    analyzer='word',
    lowercase=True,
    stop_words=text.ENGLISH_STOP_WORDS,
    max_features=100000
)
X_matrix = vectorizer.fit_transform(df['text'])

In [58]:
X_matrix[0].toarray()

array([[0, 0, 0, ..., 0, 0, 0]])

In [59]:
len(vectorizer.get_feature_names())

100000

In [17]:
vectorizer.get_feature_names()[::200]

['00',
 '1129',
 '151',
 '1894',
 '19581964',
 '1969his',
 '1977',
 '1983during',
 '1990his',
 '1995s',
 '200002',
 '2004since',
 '2010',
 '2030',
 '2530',
 '332nd',
 '43',
 '532',
 '665',
 '831',
 'aa',
 'abinanti',
 'accidental',
 'actively',
 'adjudication',
 'aerodynamicist',
 'agein',
 'aidsactivist',
 'akers',
 'alberto',
 'alexie',
 'allawis',
 'almaden',
 'altos',
 'ameche',
 'amplio',
 'andes',
 'angwin',
 'anse',
 'antonia',
 'appearing',
 'aranjuez',
 'arguing',
 'arranging',
 'asaka',
 'asmp',
 'assyrians',
 'atlantis',
 'audacious',
 'authora',
 'avis',
 'ayodhya',
 'bacharach',
 'bagher',
 'balearic',
 'bancarella',
 'baranovichi',
 'barons',
 'basically',
 'bavaria',
 'beaudoin',
 'beijing',
 'bemidji',
 'bergamini',
 'bessler',
 'bhargava',
 'bigname',
 'biographies',
 'bitchin',
 'blanchet',
 'blondinbella',
 'boardon',
 'bojan',
 'bonnier',
 'borne',
 'bousman',
 'branca',
 'breisgau',
 'bristol',
 'broten',
 'bu',
 'bullion',
 'burtch',
 'bziers',
 'calderons',
 'cam

In [60]:
idf_values = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))
idf_values

AttributeError: 'CountVectorizer' object has no attribute 'idf_'

In [61]:
vectorizer.vocabulary_

{'digby': 26542,
 'morrell': 60711,
 'born': 14798,
 '10': 41,
 'october': 65047,
 '1979': 1259,
 'australian': 9766,
 'rules': 77734,
 'footballer': 34842,
 'played': 70002,
 'kangaroos': 48165,
 'carlton': 17547,
 'football': 34839,
 'league': 52117,
 'western': 96873,
 'australia': 9752,
 'early': 29123,
 'senior': 81002,
 'west': 96844,
 'perth': 68999,
 'career': 17444,
 'falcons': 32922,
 'spanned': 84738,
 '19982000': 1879,
 'clubs': 20152,
 'leading': 52104,
 'goalkicker': 38535,
 '2000': 1994,
 'age': 5194,
 '21': 2649,
 'recruited': 74535,
 'club': 20135,
 'round': 77378,
 'selection': 80743,
 '2001': 2039,
 'afl': 5092,
 'rookie': 77140,
 'draft': 28159,
 'forward': 35134,
 'twice': 92368,
 'kicked': 49302,
 'goals': 38541,
 'time': 90041,
 'losing': 54364,
 'cause': 18032,
 'sydney': 87563,
 '2002': 2085,
 'following': 34776,
 'season': 80344,
 'drawn': 28264,
 'game': 36688,
 '2003': 2126,
 'traded': 91153,
 'david': 24843,
 'teague': 88481,
 'exchange': 32307,
 'corey': 2

## Cosine Similarity

In [24]:
cosine_similarity([[0,1,2,3]], [[2,2,0,1]])

array([[0.4454354]])

In [25]:
df[['Biden' in x for x in df['name']]]

,URI,name,text
24478,<http://dbpedia.org/resource/Joe_Biden>,Joe Biden,joseph robinette joe biden jr dosf rbnt badn b...
29590,<http://dbpedia.org/resource/Jill_Biden>,Jill Biden,jill tracy biden ne jacobs previously stevenso...


In [ ]:
index = 24478

In [27]:
df['text'][index]

'joseph robinette joe biden jr dosf rbnt badn born november 20 1942 is the 47th and current vice president of the united states jointly elected with president barack obama he is a member of the democratic party and was a united states senator from delaware from january 3 1973 until his resignation on january 15 2009 following his election to the vice presidency in 2012 biden was elected to a second term alongside obamabiden was born in scranton pennsylvania and lived there for ten years before moving to delaware he became an attorney in 1969 and was elected to the new castle county council in 1970 biden was first elected to the senate in 1972 and became the sixthyoungest senator in us history he was reelected to the senate six times and was the fourth most senior senator at the time of his resignation biden was a longtime member and former chairman of the foreign relations committee his strong advocacy helped bring about us military assistance and intervention during the bosnian war he

In [62]:
for y, x  in zip(X_matrix[index].toarray()[0], vectorizer.get_feature_names()):
    print (y, x)

0 00
0 000
0 0001
0 001
0 002
0 003
0 004
0 005
0 006
0 007
0 008
0 009
0 01
0 010
0 011
0 012
0 013
0 014
0 015
0 016
0 017
0 018
0 02
0 026
0 03
0 030
0 036
0 04
0 040
0 05
0 050
0 06
0 07
0 0708
0 071
0 075
0 077
0 08
0 09
0 093
0 0for4
0 10
2 100
0 1000
0 10000
0 100000
0 1000000
0 10000000
0 100000000
0 10000m
0 1000cc
0 1000m
0 1000th
0 1000yard
0 1001
0 100200
0 1003
0 1004
0 1005
0 1006
0 1007
0 1009
0 100e
0 100in
0 100k
0 100m
0 100meter
0 100metre
0 100mile
0 100point
0 100th
0 100yard
0 100year
0 101
0 1010
0 1011
0 1012
0 1013
0 1014
0 1015
0 1018
0 1019
0 101st
0 102
0 1020
0 1021
0 1023
0 1024
0 1025
0 1027
0 1028
0 1029
0 102nd
0 103
0 1030
0 1033
0 1034
0 1035
0 1037
0 1038
0 1039
0 103rd
0 104
0 1040
0 1041
0 1043
0 1045
0 1046
0 1047
0 1048
0 1049
0 104th
0 105
0 1050
0 105000
0 1051
0 1052
0 1053
0 1054
0 1056
0 1057
0 1058
0 1059
0 105th
0 106
0 1060
0 106000
0 1061
0 1062
0 1065
0 1067
0 106th
0 107
0 1070
0 1072
0 1075
0 1076
0 1077
0 1079
0 107th
0 108
0 1080
0 

0 19922004
0 19922005
0 19922006
0 19922007
0 199293
0 199294
0 199295
0 199296
0 199297
0 199299
0 1992after
0 1992as
0 1992at
0 1992before
0 1992born
0 1992de
0 1992during
0 1992from
0 1992he
0 1992his
0 1992in
0 1992on
0 1992s
0 1992she
0 1992since
0 1992the
0 1993
0 19931994
0 19931995
0 19931995he
0 19931996
0 19931996he
0 19931997
0 19931998
0 19931999
0 19932000
0 19932001
0 19932002
0 19932003
0 19932004
0 19932007
0 19934
0 19935
0 199394
0 199395
0 199396
0 199397
0 199398
0 199399
0 1993a
0 1993after
0 1993as
0 1993at
0 1993following
0 1993from
0 1993he
0 1993her
0 1993his
0 1993in
0 1993on
0 1993s
0 1993she
0 1993the
0 1994
0 19941995
0 19941996
0 19941997
0 19941998
0 19941999
0 19942000
0 19942001
0 19942002
0 19942003
0 19942004
0 19942005
0 19942006
0 19942010
0 19945
0 199495
0 199496
0 199497
0 199498
0 199499
0 1994a
0 1994after
0 1994as
0 1994born
0 1994during
0 1994from
0 1994he
0 1994his
0 1994in
0 1994on
0 1994s
0 1994she
0 1994since
0 1994the
0 1995
0 19951996
0

0 432
0 433
0 434
0 435
0 436
0 437
0 438
0 439
0 43rd
0 44
0 440
0 4400
0 44000
0 441
0 442
0 4421
0 443
0 444
0 445
0 446
0 447
0 448
0 449
0 44th
0 45
0 450
0 4500
0 45000
0 450000
0 451
0 452
0 453
0 454
0 455
0 456
0 457
0 458
0 459
0 45minute
0 45rpm
0 45s
0 45th
0 45yearold
0 46
0 460
0 4600
0 460000
0 461
0 462
0 463
0 464
0 465
0 466
0 46664
0 467
0 468
0 46th
0 47
0 470
0 4700
0 472
0 473
0 474
0 475
0 476
0 477
0 478
0 479
0 47th
0 48
0 480
0 4800
0 481
0 482
0 483
0 484
0 485
0 486
0 487
0 488
0 489
0 48hour
0 48km
0 48th
0 49
0 491
0 492
0 493
0 494
0 495
0 496
0 497
0 498
0 499
0 49er
0 49ers
0 49th
0 4a
0 4ad
0 4b
0 4c
0 4d
0 4for4
0 4g
0 4h
0 4he
0 4in
0 4k
0 4kids
0 4m
0 4man
0 4part
0 4pm
0 4s
0 4stroke
0 4talent
0 4th
0 4the
0 4time
0 4x
0 4x100
0 4x100m
0 4x200m
0 4x4
0 4x400
0 4x400m
0 4year
0 4yearold
1 50
0 500
0 5000
0 50000
0 500000
0 5000000
0 5000m
0 5000metre
0 5000th
0 500cc
0 500m
0 500s
0 500th
0 501
0 501c3
0 502
0 503
0 504
0 5044
0 505
0 5050
0 5054
0 

0 adalgisa
0 adam
0 adam12
0 adama
0 adamant
0 adamawa
0 adamek
0 adami
0 adamich
0 adamis
0 adamonis
0 adamou
0 adamov
0 adams
0 adamsdale
0 adamsin
0 adamson
0 adamss
0 adamu
0 adan
0 adana
0 adani
0 adapt
0 adaptability
0 adaptable
0 adaptation
0 adaptations
0 adapted
0 adapting
0 adaption
0 adaptions
0 adaptive
0 adapts
0 adb
0 adbs
0 adbusters
0 adc
0 adcock
0 add
0 addabbo
0 addams
0 added
0 addeens
0 adderley
0 addict
0 addicted
0 addiction
0 addictionin
0 addictions
0 addictive
0 addicts
0 addie
0 adding
0 addington
0 addis
0 addison
0 addisonwesley
0 addition
0 additional
0 additionally
0 additions
0 additive
0 additives
0 address
0 addressed
0 addresses
0 addressing
0 adds
0 addu
0 addy
0 ade
0 adeeb
0 adel
0 adelaide
0 adelaidehe
0 adelaides
0 adele
0 adeleke
0 adeles
0 adeline
0 adelphi
0 adelson
0 adema
0 ademi
0 aden
0 adenaike
0 adept
0 adeptness
0 adequacy
0 adequate
0 adequately
0 ader
0 adere
0 aderem
0 adesida
0 adetona
0 adevantgarde
0 adewoye
0 adey
0 adeyemi
0 adh

0 ajulubushell
0 ajumogobia
0 ajyuc
0 ak
0 ak47
0 ak47s
0 aka
0 akademi
0 akademia
0 akademicka
0 akademie
0 akademik
0 akademio
0 akademisk
0 akademy
0 akagawa
0 akai
0 akaka
0 akalaitis
0 akamai
0 akan
0 akane
0 akasaka
0 akash
0 akashi
0 akashic
0 akatsuka
0 akayev
0 akayeva
0 akbar
0 akbayan
0 ake
0 akebono
0 akeelah
0 akel
0 akele
0 akena
0 aker
0 akerman
0 akers
0 akershus
0 akgn
0 akhbar
0 akher
0 akhil
0 akhisar
0 akhondy
0 akhoon
0 akhtar
0 akhtars
0 aki
0 akiba
0 akif
0 akihiro
0 akihito
0 akiko
0 akim
0 akin
0 aking
0 akinlonu
0 akinola
0 akinori
0 akins
0 akintunde
0 akio
0 akira
0 akiskal
0 akister
0 akita
0 akiva
0 akiwenziedamm
0 akiyama
0 akiyo
0 akiyoshi
0 akkadian
0 akkaya
0 akker
0 akkerman
0 akkitham
0 akko
0 akl
0 aklaff
0 akn
0 ako
0 akoitai
0 akol
0 akolo
0 akolor
0 akon
0 akonobi
0 akopian
0 akovec
0 akp
0 akram
0 akromiya
0 akron
0 akronfamily
0 aksarben
0 akshay
0 akshaya
0 aksoy
0 akta
0 aktion
0 aku
0 akuraters
0 akure
0 akureyri
0 akut
0 akutagawa
0 akwa
0 

...

0 vibe
0 vibes
0 vibhushan
0 viborg
0 vibrancy
0 vibrant
0 vibraphone
0 vibraphonist
0 vibration
0 vibrations
0 vibrato
0 vic
0 vicar
0 vicari
0 vicario
0 vice
0 viceadmiral
0 vicecaptain
0 vicechair
0 vicechairman
0 vicechairperson
0 vicechampion
0 vicechancellor
0 vicechancellors
0 vicedean
0 vicedirector
0 vicegovernor
0 vicehead
0 vicemarshal
0 vicemayor
0 viceminister
0 vicente
0 vicenza
0 vicenzo
0 vicepremier
0 vicepresidency
0 vicepresident
0 vicepresidential
0 vicepresidents
0 viceprincipal
0 viceprovost
0 vicerector
0 viceregal
0 viceroy
0 vices
0 vicesecretary
0 vicespeaker
0 viceversa
0 vichy
0 vicinity
0 vicious
0 viciously
0 vick
0 vickers
0 vicki
0 vickie
0 vicksburg
0 vicky
0 vico
0 vics
0 victim
0 victimisation
0 victimization
0 victimized
0 victims
0 victo
0 victoire
0 victoires
0 victor
1 victoria
0 victoriahe
0 victoriain
0 victorian
0 victorianaustralian
0 victorians
0 victorias
0 victoriaville
0 victories
0 victorious
0 victors
0 victorvictoria
0 victory
0 victory

0 wac
0 wachner
0 wachovia
0 wachs
0 wachtel
0 wachtell
0 wachuku
0 wack
0 wacken
0 wacker
0 wackers
0 wacky
0 waco
0 wad
0 wada
0 wadada
0 wadavati
0 waddell
0 waddle
0 wade
0 wadegiles
0 wadel
0 wades
0 wadham
0 wadi
0 wadih
0 wading
0 wadler
0 wadley
0 wadsworth
0 wadysaw
0 waechter
0 wael
0 wafb
0 wafer
0 waffen
0 waffenss
0 waffle
0 wafl
0 wag
0 wage
0 waged
0 wagenfuhr
0 wageningen
0 wagenstein
0 wager
0 wagering
0 wagers
0 wages
0 wagga
0 wagh
0 waging
0 wagner
0 wagnerian
0 wagnerpasquier
0 wagners
0 wagon
0 wagoner
0 wagons
0 wagstaff
0 wah
0 wahab
0 wahba
0 wahhab
0 wahid
0 wahine
0 wahl
0 wahlberg
0 wahrheit
0 wahshee
0 wahwah
0 wai
0 waibel
0 waiheke
0 waikato
0 waikiki
0 waiko
0 wailers
0 waimea
0 waimnalo
0 wain
0 wainthropp
0 wainwright
0 wainwrights
0 wais
0 waist
0 wait
0 waitakere
0 waitangi
0 waite
0 waited
0 waiter
0 waiters
0 waites
0 waitin
0 waiting
0 waitress
0 waitresses
0 waits
0 waive
0 waived
0 waiver
0 waivers
0 wajiha
0 wak
0 waka
0 wakabayashi
0 wakamatsu

0 wilkie
0 wilkin
0 wilkins
0 wilkinson
0 wilkinsons
0 wilkosz
0 wilkow
0 wilks
0 willa
0 willamette
0 willan
0 willard
0 willardlewis
0 willcox
0 willem
0 willemalexander
0 willems
0 willen
0 willenhall
0 willes
0 willesden
0 willett
0 willetts
0 willey
0 willhite
0 willhoite
0 willi
0 william
0 williams
0 williamsburg
0 williamshe
0 williamsin
0 williamson
0 williamsons
0 williamsport
0 williamss
0 williamsthe
0 williamstown
0 williamsville
0 willie
0 willies
0 willighagen
0 willin
0 willing
0 willingboro
0 willingly
0 willingness
0 willis
0 willison
0 willmar
0 willmott
0 willner
0 willoch
0 willochs
0 willoughby
0 willow
0 willows
0 willpower
0 wills
0 willson
0 willumstad
0 willy
0 wilma
0 wilmer
0 wilmette
0 wilmington
0 wilmot
0 wilmott
0 wilms
0 wilmshurst
0 wilmslow
0 wilsbach
0 wilshire
0 wilson
0 wilsonraybould
0 wilsons
0 wilt
0 wilton
0 wiltshire
0 wiltzie
0 wily
0 wim
0 wimax
0 wimbledon
0 wimbledonin
0 wimbleton
0 wimbush
0 wimer
0 wimmer
0 wimp
0 wimpy
1 win
0 winans
0 

0 yearending
0 yearevans
0 yearfollowing
0 yearfor
0 yearfrom
0 yearhaving
0 yearhe
0 yearher
0 yearhis
0 yearin
0 yearit
0 yearlong
0 yearly
0 yearning
0 yearold
0 yearolds
0 yearon
0 yearprior
0 yearround
1 years
0 yearsa
0 yearsafter
0 yearsalso
0 yearsamong
0 yearsand
0 yearsas
0 yearsat
0 yearsborn
0 yearsbrown
0 yearsdr
0 yearsdue
0 yearsduring
0 yearsfollowing
0 yearsfor
0 yearsfrom
0 yearshe
0 yearsher
0 yearshis
0 yearsi
0 yearsin
0 yearsince
0 yearsit
0 yearslong
0 yearsmr
0 yearsold
0 yearson
0 yearsone
0 yearsother
0 yearsprior
0 yearsshe
0 yearssince
0 yearsthe
0 yearswhile
0 yearswith
0 yearsworking
0 yearthe
0 yearwhen
0 yearwhile
0 yearwood
0 yeast
0 yeats
0 yeatss
0 yeboah
0 yechiel
0 yeddyurappa
0 yediot
0 yedioth
0 yee
0 yees
0 yeeun
0 yefim
0 yefimov
0 yefimovich
0 yefimovs
0 yefremov
0 yeginobali
0 yegorova
0 yeh
0 yehezkel
0 yehia
0 yehoshua
0 yehuda
0 yehudi
0 yehudit
0 yekaterina
0 yekaterinburg
0 yekimov
0 yelena
0 yelich
0 yelizaveta
0 yell
0 yelled
0 yelling


0 zola
0 zolder
0 zolezzi
0 zoli
0 zolinsky
0 zolkwer
0 zollitsch
0 zoltan
0 zoltn
0 zomba
0 zombie
0 zombies
0 zona
0 zonal
0 zondervan
0 zone
0 zoned
0 zones
0 zoning
0 zonjic
0 zonn
0 zoo
0 zooey
0 zookeeper
0 zoological
0 zoologist
0 zoology
0 zoom
0 zoomusicology
0 zoos
0 zoot
0 zophres
0 zora
0 zoran
0 zorba
0 zordan
0 zorkin
0 zorn
0 zornberg
0 zornotza
0 zorns
0 zoroastrian
0 zorro
0 zos
0 zosha
0 zosterops
0 zotowski
0 zouk
0 zoxea
0 zpa
0 zrcher
0 zrenjanin
0 zrich
0 zro
0 zs
0 zschech
0 zsolt
0 zstar
0 zsuzsanna
0 ztbl
0 ztrk
0 zu
0 zubair
0 zubaydah
0 zubers
0 zubin
0 zucchero
0 zucker
0 zuckerberg
0 zuckerman
0 zuckermann
0 zug
0 zugibe
0 zuid
0 zukerman
0 zuleyma
0 zulfikar
0 zulfiqar
0 zulhasnan
0 zulia
0 zulu
0 zululand
0 zum
0 zuma
0 zuniga
0 zupan
0 zupcu
0 zur
0 zurab
0 zurek
0 zurenuoc
0 zuri
0 zurich
0 zurichs
0 zus
0 zusman
0 zuuk
0 zuzana
0 zvezda
0 zvi
0 zvonimir
0 zvornik
0 zwaig
0 zwart
0 zwarts
0 zwei
0 zweig
0 zweli
0 zwickau
0 zwikker
0 zwolle
0 zwolsman
0 

In [41]:

print(df['name'][index])
main = X_matrix[index]

df['score'] = cosine_similarity(main, X_matrix)[0]

Joe Biden


In [42]:
df.sort_values(by='score',ascending=False).head()

,URI,name,text,score
24478,<http://dbpedia.org/resource/Joe_Biden>,Joe Biden,joseph robinette joe biden jr dosf rbnt badn b...,1.000000
29590,<http://dbpedia.org/resource/Jill_Biden>,Jill Biden,jill tracy biden ne jacobs previously stevenso...,0.472481
16880,<http://dbpedia.org/resource/Cynthia_Hogan>,Cynthia Hogan,cynthia c hogan born cincinnati ohio about 195...,0.407093
35811,<http://dbpedia.org/resource/Barack_Obama>,Barack Obama,barack hussein obama ii brk husen bm born augu...,0.339517
23309,<http://dbpedia.org/resource/Chris_Coons>,Chris Coons,christopher andrew chris coons born september ...,0.273705


In [43]:
df.sort_values(by='score',ascending=False).head()['text'].values

array(['joseph robinette joe biden jr dosf rbnt badn born november 20 1942 is the 47th and current vice president of the united states jointly elected with president barack obama he is a member of the democratic party and was a united states senator from delaware from january 3 1973 until his resignation on january 15 2009 following his election to the vice presidency in 2012 biden was elected to a second term alongside obamabiden was born in scranton pennsylvania and lived there for ten years before moving to delaware he became an attorney in 1969 and was elected to the new castle county council in 1970 biden was first elected to the senate in 1972 and became the sixthyoungest senator in us history he was reelected to the senate six times and was the fourth most senior senator at the time of his resignation biden was a longtime member and former chairman of the foreign relations committee his strong advocacy helped bring about us military assistance and intervention during the bosnian

In [44]:
df.head()

,URI,name,text,score
0,<http://dbpedia.org/resource/Digby_Morrell>,Digby Morrell,digby morrell born 10 october 1979 is a former...,0.010675
1,<http://dbpedia.org/resource/Alfred_J._Lewy>,Alfred J. Lewy,alfred j lewy aka sandy lewy graduated from un...,0.006669
2,<http://dbpedia.org/resource/Harpdog_Brown>,Harpdog Brown,harpdog brown is a singer and harmonica player...,0.009943
3,<http://dbpedia.org/resource/Franz_Rottensteiner>,Franz Rottensteiner,franz rottensteiner born in waidmannsfeld lowe...,0.013306
4,<http://dbpedia.org/resource/G-Enka>,G-Enka,henry krvits born 30 december 1974 in tallinn ...,0.001138


In [45]:
df = pd.read_csv('./data/people_wiki.csv')

In [46]:
df.head()

,URI,name,text
0,<http://dbpedia.org/resource/Digby_Morrell>,Digby Morrell,digby morrell born 10 october 1979 is a former...
1,<http://dbpedia.org/resource/Alfred_J._Lewy>,Alfred J. Lewy,alfred j lewy aka sandy lewy graduated from un...
2,<http://dbpedia.org/resource/Harpdog_Brown>,Harpdog Brown,harpdog brown is a singer and harmonica player...
3,<http://dbpedia.org/resource/Franz_Rottensteiner>,Franz Rottensteiner,franz rottensteiner born in waidmannsfeld lowe...
4,<http://dbpedia.org/resource/G-Enka>,G-Enka,henry krvits born 30 december 1974 in tallinn ...


In [48]:
index = df[df['name'] == 'David Beckham'].index[0]

X = X_matrix[ index ]

values = cosine_similarity( X, X_matrix)[0]
ps_values = pd.Series(data=values, index=df.index)


In [49]:
ps_values.sort_values(ascending=False, inplace=True)
ps_values

23386    1.000000
24913    0.278273
26756    0.249382
38666    0.238645
24258    0.234768
           ...   
20586    0.000000
9550     0.000000
28649    0.000000
33741    0.000000
4725     0.000000
Length: 42786, dtype: float64

In [50]:
df.loc[ps_values.index]

,URI,name,text
23386,<http://dbpedia.org/resource/David_Beckham>,David Beckham,david robert joseph beckham obe bkm born 2 may...
24913,<http://dbpedia.org/resource/Bobby_Charlton>,Bobby Charlton,sir robert bobby charlton cbe born 11 october ...
26756,<http://dbpedia.org/resource/Wayne_Rooney>,Wayne Rooney,wayne mark rooney runi born 24 october 1985 is...
38666,<http://dbpedia.org/resource/Shay_Given>,Shay Given,shay john james given born 20 april 1976 is an...
24258,<http://dbpedia.org/resource/Sol_Campbell>,Sol Campbell,sulzeer jeremiah sol campbell born 18 septembe...
...,...,...,...
20586,<http://dbpedia.org/resource/Mohammed_Faizal_P...,Mohammed Faizal P. P.,pp mohammed faizal is an indian politician bel...
9550,<http://dbpedia.org/resource/Peter_Hobson>,Peter Hobson,r p hobson or peter hobson is a professor of d...
28649,<http://dbpedia.org/resource/Jesper_Mogensen>,Jesper Mogensen,jesper mogensen is a danish neuroscientist who...
33741,<http://dbpedia.org/resource/Yossi_Matias>,Yossi Matias,yossi matias is an israeli computer scientist ...


In [51]:
def find_similar_people(name):
    indexes = df[df['name'] == name].index
    if len(indexes) == 0:
        print('no one found')
        return
    
    index = indexes[0]
    X = X_matrix[ index ]

    values = cosine_similarity( X, X_matrix)[0]
    ps_values = pd.Series(data=values, index=df.index)
    ps_values.sort_values(ascending=False, inplace=True)

    return df.loc[ps_values.index]

In [52]:
find_similar_people('Tim Burton')

,URI,name,text
2001,<http://dbpedia.org/resource/Tim_Burton>,Tim Burton,timothy walter tim burton brtn born august 25 ...
18180,<http://dbpedia.org/resource/Helena_Bonham_Car...,Helena Bonham Carter,helena bonham carter cbe born 26 may 1966 is a...
13178,<http://dbpedia.org/resource/Kristen_Stewart>,Kristen Stewart,kristen jaymes stewart born april 9 1990 is an...
34701,<http://dbpedia.org/resource/Amanda_Seyfried>,Amanda Seyfried,amanda michelle seyfried safrd syfred born dec...
2163,<http://dbpedia.org/resource/M._K._Hobson>,M. K. Hobson,m k hobson born january 21 1969 is a speculati...
...,...,...,...
34711,<http://dbpedia.org/resource/Frank_Walker_(Aus...,Frank Walker (Australian rules footballer),frank dickie walker was an australian rules fo...
38984,<http://dbpedia.org/resource/Bruce_Lahn>,Bruce Lahn,bruce lahn is the william b graham professor o...
34747,<http://dbpedia.org/resource/Anna_Vainikka>,Anna Vainikka,anne vainikka is a linguist specialising in th...
16485,<http://dbpedia.org/resource/Tamar_Ross>,Tamar Ross,tamar ross is a professor of jewish philosophy...


In [54]:
df[ df['name'].str.contains('Depp')]

,URI,name,text
